# Яндекс.Музыка

Сравнение Москвы и Петербурга окружено мифами. Например:
 * Москва — мегаполис, подчинённый жёсткому ритму рабочей недели;
 * Петербург — культурная столица, со своими вкусами.

На данных Яндекс.Музыки вы сравните поведение пользователей двух столиц.

**Цель исследования** — проверьте три гипотезы:
1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города. 
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

**Ход исследования**

Данные о поведении пользователей вы получите из файла `yandex_music_project.csv`. О качестве данных ничего не известно. Поэтому перед проверкой гипотез понадобится обзор данных. 

Вы проверите данные на ошибки и оцените их влияние на исследование. Затем, на этапе предобработки вы поищете возможность исправить самые критичные ошибки данных.
 
Таким образом, исследование пройдёт в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.



## Обзор данных

In [53]:
# импорт библиотек
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
# чтение файла с данными
df = pd.read_csv('downloads/df1.csv')

In [3]:
df.head(10)

,Unnamed: 0,userID,Track,artist,genre,City,time,Day
0,0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  65079 non-null  int64 
 1     userID    65079 non-null  object
 2   Track       63848 non-null  object
 3   artist      57876 non-null  object
 4   genre       63881 non-null  object
 5     City      65079 non-null  object
 6   time        65079 non-null  object
 7   Day         65079 non-null  object
dtypes: int64(1), object(7)
memory usage: 4.0+ MB


**Выводы**

В каждой строке таблицы — данные о прослушанном треке. Часть колонок описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку. 

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных, а в названиях колонок — расхождения с хорошим стилем.

Необходимо устранить проблемы в данных.

## Предобработка данных

In [5]:
# перечень названий столбцов таблицы df
df.columns

Index(['Unnamed: 0', '  userID', 'Track', 'artist', 'genre', '  City  ',
       'time', 'Day'],
      dtype='object')

In [6]:
#удалим лишний столбец
df.drop('Unnamed: 0', axis=1)

#переименование столбцов
df = df.rename(columns = {'  userID': 'user_id', 
                          'Track': 'track', 
                          '  City  ': 'city', 
                          'Day': 'day'})

In [7]:
#посчитаем количество пропусков в столбцах:
df.isna().sum()

Unnamed: 0       0
user_id          0
track         1231
artist        7203
genre         1198
city             0
time             0
day              0
dtype: int64

In [8]:
# перебор названий столбцов в цикле и замена пропущенных значений на 'unknown'
columns_to_replace = ['track', 'artist', 'genre']
for item in columns_to_replace:
    df[item] = df[item].fillna('unknown')

In [9]:
df.isna().sum()# подсчёт пропусков

Unnamed: 0    0
user_id       0
track         0
artist        0
genre         0
city          0
time          0
day           0
dtype: int64

Пропусков в таблице не осталось. Провеим данные на дубликаты:

In [67]:
df.duplicated().sum()

0

Дубликатов в данных нет. Проверим, какие жанры представлены в датасете:

In [11]:
genre_sort = df['genre'].sort_values()
genre_sort.unique()# Просмотр уникальных названий жанров

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

In [68]:
#устраним проблемы в данных: объединим хип-хоп в одну категорию:
duplicates = ['hip', 'hop', 'hip-hop']
name = 'hiphop'
df['genre'] = df['genre'].replace(duplicates, name)

#соберем электронную музыку в одну категорию:
df.loc[df['genre'] == 'электроника', 'genre'] = 'electronic'

#заменим жанр "ïîï"на uncnown:
df.loc[df['genre'] == 'ïîï', 'genre'] = 'unknown'

In [69]:
#проверим, все ли проблемы устранены:
genre_sort = df['genre'].sort_values()
genre_sort.unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

**Выводы**

Предобработка обнаружила три проблемы в данных:

- нарушения в стиле заголовков,
- пропущенные значения,
- наличие дубликатов.

Пропущенные значения были заменены на 'unknown'.

Теперь можно перейти к проверке гипотез.

## Проверка гипотез

### Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверим это предположение по данным о трёх днях недели — понедельнике, среде и пятнице.

In [70]:
#подсчитаем общее количество прослушиваний треков по городам:
df.groupby('city')['user_id'].count()

city
Moscow              45360
Saint-Petersburg    19719
Name: user_id, dtype: int64

В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.

In [15]:
#подсчитаем общее количество прослушиваний подням недели:
df.groupby('day')['user_id'].count()

day
Friday       23149
Monday       22697
Wednesday    19233
Name: user_id, dtype: int64

В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

In [55]:
#сгруппируем данные по городу и дню недели:
data_pivot = df.groupby(['city', 'day'])['user_id'].count().reset_index()

data_pivot

,city,day,user_id
0,Moscow,Friday,16890
1,Moscow,Monday,16715
2,Moscow,Wednesday,11755
3,Saint-Petersburg,Friday,6259
4,Saint-Petersburg,Monday,5982
5,Saint-Petersburg,Wednesday,7478


In [71]:
#построим график прослушиваний по дням недели и городам:
fig = px.bar(data_pivot, x='day', y='user_id',
             color='city', barmode = 'group', width=900, height=500)
fig.update_layout(title='Распределение количества прослушиваний по дням недели и городам',
                  yaxis_title='Количество прослушиваний',
                  xaxis_title='День недели')
fig.show();

**Выводы**

Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы.

### Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

In [64]:
#отфильтруем данные по Москве:
moscow_general = df[df['city'] == 'Moscow']

In [65]:
#отфильтруем данные по Санкт-Петербургу:
spb_general = df[df['city'] == 'Saint-Petersburg']

In [27]:
#напишем функцию, считающую количество прослушиваний треков по жанрам в нужный промежуток времени:
def genre_weekday(df, day, time1, time2):
    genre_df = df[df['day'] == day]
    genre_df = genre_df[genre_df['time'] < time2]
    genre_df = genre_df[genre_df['time'] > time1]
    genre_df_grouped = genre_df.groupby('genre')['genre'].count()
    genre_df_sorted = genre_df_grouped.sort_values(ascending=False)
    return genre_df_sorted[:10]

Cравните результаты функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [66]:
#посчитаем, сколько раз прослушивали треки с 7 до 11 утра в Москве в понедельник:
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

genre
pop            830
dance          589
rock           511
electronic     501
hiphop         311
ruspop         203
world          190
rusrap         188
alternative    175
unknown        172
Name: genre, dtype: int64

In [29]:
#посчитаем, сколько раз прослушивали треки с 7 до 11 утра в Санкт-Петербурге в понедельник:
genre_weekday(spb_general, 'Monday', '07:00', '11:00')

genre
pop            238
dance          192
rock           173
electronic     154
hiphop          88
ruspop          68
alternative     65
rusrap          56
jazz            47
classical       42
Name: genre, dtype: int64

In [30]:
#посчитаем, сколько раз прослушивали треки с 17 до 23 часов в Москве в пятницу:
genre_weekday(moscow_general, 'Friday', '17:00', '23:00')

genre
pop            761
rock           546
dance          521
electronic     510
hiphop         282
world          220
ruspop         184
alternative    176
classical      171
rusrap         151
Name: genre, dtype: int64

In [31]:
#посчитаем, сколько раз прослушивали треки с 17 до 23 часов в Санкт-Петербурге в пятницу:
genre_weekday(spb_general, 'Friday', '17:00', '23:00')

genre
pop            279
rock           230
electronic     227
dance          221
hiphop         103
alternative     67
jazz            66
rusrap          66
classical       64
world           60
Name: genre, dtype: int64

**Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

In [72]:
#сгруппируем данные по Москве по жанрам:
moscow_genres = moscow_general.groupby('genre')['genre'].count()
 
moscow_genres = moscow_genres.sort_values(ascending=False)

In [73]:
#посмотрим, какие 10 жанров наиболее популярны в Москве:
moscow_genres.head(10)

genre
pop            6253
dance          4707
rock           4188
electronic     4010
hiphop         2215
classical      1712
world          1516
alternative    1466
ruspop         1453
rusrap         1239
Name: genre, dtype: int64

In [34]:
#сгруппируем данные по Санкт-Петербургу по жанрам:
spb_genres = spb_general.groupby('genre')['genre'].count()

spb_genres = spb_genres.sort_values(ascending=False)

In [35]:
#посмотрим, какие 10 жанров наиболее популярны в Санкт-Петербурге:
spb_genres.head(10)

genre
pop            2597
dance          2054
rock           2004
electronic     1843
hiphop         1020
alternative     700
classical       684
rusrap          604
ruspop          565
world           553
Name: genre, dtype: int64

**Выводы**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге. 


## Итоги исследования

Вы проверили три гипотезы и установили:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге. 

Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.